Activate conda w251_project3
Four videos were not cropped correctly in 0_create_model_data.ipynb.  
* ASL_2006_10_10_scene3-camera1_BASEBALL_v1_c1_train.mov
* ASL_2006_10_10_scene3-camera1_AWKWARD_v0_c4_test.mov
* ASL_2007_05_24_scene12-camera1_DRESS_v0_c18_train.mov
* ASL_2007_05_24_scene12-camera1_ENOUGH_v1_c5_train.mov

I made a copy of `train_test_videos_NEW` folder and recreated the videos above.

In [6]:
import cv2
import os
import urllib
import pandas as pd
import re
import numpy as np
from matplotlib import pyplot as plt

In [2]:
project_path = r'D:\UCBerkeley\CourseWork\202001\W251\Homework\Project'
excel_name = 'SignVideoInventory.xlsx'
excel_sheet_name = 'select19_input'
video_url_prefix = 'http://csr.bu.edu/ftp/asl/asllvd/asl-data2/quicktime/'
dir_original_video = 'original_videos'
dir_trimmed_video = 'trimmed_videos_NEW'
# dir_train = 'train_videos_NEW'
# dir_test = 'test_videos_NEW'
dir_train_test = 'train_test_videos_NEW3'
test_pct = 0.2

In [3]:
####################### DO NOT NEED TO RERUN THIS CELL #######################
# Read Excel file 
inventory_df = pd.read_excel(os.path.join(project_path, excel_name), sheet_name=excel_sheet_name)
# Construct video names
inventory_video_name_df = inventory_df['Session'].map(str) + '/scene' \
                          + inventory_df['Scene'].map(str) + '-camera1.mov'
# Get a list of all videos
inventory_video_name_lst = inventory_video_name_df.tolist()
# Get a list of unique videos
inventory_video_name_set = set(inventory_video_name_lst)
# Get a list of tuples with start and end frames
inventory_start_end_frame_lst = inventory_df[['Start', 'End']].apply(tuple, axis=1).tolist()
# Get a list of words
inventory_word_lst = inventory_df['Main New Gloss'].tolist()
# Get a list of variants
inventory_variant_lst = inventory_df['Variant'].tolist()

# Sanity check
# 95 original videos should be downloaded
print(len(inventory_video_name_set))

95


In [4]:
####################### DO NOT NEED TO RERUN THIS CELL #######################
# def download_original_videos(p_path, out_dir, video_name_set):
#     for video_name in video_name_set:
#         # Use regular expression to replace / with _
#         saved_video_name = re.sub('/', '_', video_name)
#         # Download only if it doesn't exist yet
#         print(saved_video_name)
#         if not os.path.isfile(os.path.join(p_path, out_dir, saved_video_name)):
#             # Create an URL to download each video
#             video_url = video_url_prefix + video_name
#             # Download videos
#             urllib.request.urlretrieve(video_url, os.path.join(p_path, out_dir, saved_video_name)) 
# 
# download_original_videos(project_path, dir_original_video, inventory_video_name_set)

In [3]:
####################### DO NOT NEED TO RERUN THIS CELL #######################
def trim_one_video (p_path, in_dir, in_video, out_dir, out_video, start_frame, end_frame):
    cap = cv2.VideoCapture(os.path.join(p_path, in_dir, in_video))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width_frame  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height_frame = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) 
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(os.path.join(p_path, out_dir, out_video),fourcc, fps, (width_frame, height_frame))
    counter_frame = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # frame = cv2.flip(frame,0)
            # write the flipped frame
            if start_frame <= counter_frame <= end_frame:
                out.write(frame)
                # cv2.imshow('Preview',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            counter_frame += 1
        else:
            break
    # Release everything if job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [6]:
####################### DO NOT NEED TO RERUN THIS CELL #######################
def trim_all_videos(p_path, in_dir, out_dir, word_lst, video_name_lst, start_end_frame_lst, variant_lst):
    out_video_name_lst = []
    previous_word = ''
    counter = 0
    for word, video_name, (start_frame, end_frame), variant \
            in zip(word_lst, video_name_lst, start_end_frame_lst, variant_lst):
        if word == previous_word:
            counter += 1
        else:
            counter = 0
        original_video_name = re.sub('/', '_', video_name)
        # Construct saved video name: ASL_2008_01_11_scene2-camera1_[word]_v[variant]_[counter].mov
        new_video_name = re.sub('/', '_', video_name)[:-4] + '_' + word \
                           + '_v' + str(variant) + '_c' + str(counter) + '.mov'
        previous_word = word
        print(new_video_name)
        out_video_name_lst.append(new_video_name)
        if not os.path.isfile(os.path.join(p_path, out_dir, new_video_name)):
            trim_one_video (p_path, in_dir, original_video_name, out_dir, new_video_name, start_frame, end_frame)
    return out_video_name_lst
    
trimmed_video_name_lst = trim_all_videos(project_path, dir_original_video, dir_trimmed_video, 
                                       inventory_word_lst, inventory_video_name_lst, 
                                       inventory_start_end_frame_lst, inventory_variant_lst)

ASL_2008_01_11_scene9-camera1_AGAIN_v0_c0.mov
ASL_2008_01_11_scene9-camera1_AGAIN_v2_c1.mov
ASL_2008_05_12a_scene8-camera1_AGAIN_v0_c2.mov
ASL_2008_05_12a_scene8-camera1_AGAIN_v1_c3.mov
ASL_2008_08_04_scene7-camera1_AGAIN_v0_c4.mov
ASL_2008_08_04_scene7-camera1_AGAIN_v2_c5.mov
ASL_2011_06_08_Brady_scene9-camera1_AGAIN_v0_c6.mov
ASL_2011_06_14_Brady_scene2-camera1_AGAIN_v3_c7.mov
ASL_2011_07_19_Brady_scene116-camera1_AGAIN_v0_c8.mov
ASL_2006_10_10_scene2-camera1_AGAIN_v0_c9.mov
ASL_2007_05_24_scene5-camera1_AGAIN_v0_c10.mov
ASL_2008_01_11_scene11-camera1_ALL_v0_c0.mov
ASL_2008_05_12a_scene9-camera1_ALL_v0_c1.mov
ASL_2008_08_04_scene8-camera1_ALL_v0_c2.mov
ASL_2011_06_08_Brady_scene10-camera1_ALL_v0_c3.mov
ASL_2006_10_10_scene2-camera1_ALL_v0_c4.mov
ASL_2007_05_24_scene5-camera1_ALL_v0_c5.mov
ASL_2008_01_11_scene27-camera1_AWKWARD_v0_c0.mov
ASL_2008_05_12a_scene18-camera1_AWKWARD_v0_c1.mov
ASL_2008_08_04_scene18-camera1_AWKWARD_v0_c2.mov
ASL_2011_06_08_Brady_scene14-camera1_AWKWARD_v0_c3

In [7]:
####################### DO NOT NEED TO RERUN THIS CELL #######################
def manual_stratified_sampling(word_lst, video_lst, t_pct):
    # Default: all videos are for training, will switch x% to test later
    out_train_test_dict = dict.fromkeys(video_lst , 'train')
    video_by_word_lst = []
    previous_word = ''
    for (word, video) in zip(word_lst, video_lst):
        if word == previous_word:
            video_by_word_lst.append(video)
        else:
            # If video_by_word_lst is empty, don't perform the following steps
            # This is to handle the first iteration of the loop
            if len(video_by_word_lst) != 0:
                if len(video_by_word_lst) == 1:
                    print('ALERT: only one video for', previous_word)
                # For a given word, calculate the number of videos for validation, floored at 1 
                test_count = max(int(len(video_by_word_lst)*t_pct), 1)
                video_by_word_test_lst = np.random.choice(video_by_word_lst, test_count, replace=False)
                for i in video_by_word_test_lst:
                    out_train_test_dict[i] = 'test'
            video_by_word_lst = [video]
            previous_word = word
    # Handle the last word      
    if len(video_by_word_lst) == 1:
        print('ALERT: only one video for', previous_word)
    test_count = int(len(video_by_word_lst)*t_pct)
    video_by_word_test_lst = np.random.choice(video_by_word_lst, test_count, replace=False)
    for i in video_by_word_test_lst:
        out_train_test_dict[i] = 'val'
        
    return out_train_test_dict

video_train_test_dict = manual_stratified_sampling(inventory_word_lst, trimmed_video_name_lst, test_pct)

In [8]:
####################### DO NOT NEED TO RERUN THIS CELL #######################
video_train_test_dict



{'ASL_2008_01_11_scene9-camera1_AGAIN_v0_c0.mov': 'train',
 'ASL_2008_01_11_scene9-camera1_AGAIN_v2_c1.mov': 'train',
 'ASL_2008_05_12a_scene8-camera1_AGAIN_v0_c2.mov': 'train',
 'ASL_2008_05_12a_scene8-camera1_AGAIN_v1_c3.mov': 'test',
 'ASL_2008_08_04_scene7-camera1_AGAIN_v0_c4.mov': 'train',
 'ASL_2008_08_04_scene7-camera1_AGAIN_v2_c5.mov': 'train',
 'ASL_2011_06_08_Brady_scene9-camera1_AGAIN_v0_c6.mov': 'test',
 'ASL_2011_06_14_Brady_scene2-camera1_AGAIN_v3_c7.mov': 'train',
 'ASL_2011_07_19_Brady_scene116-camera1_AGAIN_v0_c8.mov': 'train',
 'ASL_2006_10_10_scene2-camera1_AGAIN_v0_c9.mov': 'train',
 'ASL_2007_05_24_scene5-camera1_AGAIN_v0_c10.mov': 'train',
 'ASL_2008_01_11_scene11-camera1_ALL_v0_c0.mov': 'train',
 'ASL_2008_05_12a_scene9-camera1_ALL_v0_c1.mov': 'train',
 'ASL_2008_08_04_scene8-camera1_ALL_v0_c2.mov': 'test',
 'ASL_2011_06_08_Brady_scene10-camera1_ALL_v0_c3.mov': 'train',
 'ASL_2006_10_10_scene2-camera1_ALL_v0_c4.mov': 'train',
 'ASL_2007_05_24_scene5-camera1_ALL_v

In [9]:
####################### DO NOT NEED TO RERUN THIS CELL #######################
# Save training and validation videos in separate folders
def save_train_test(p_path, in_dir, out_train_dir, out_test_dir, in_train_test_dict):
    for k, v in in_train_test_dict.items():
        if v == 'train':
            if not os.path.isfile(os.path.join(p_path, out_train_dir, k)):
                !copy {os.path.join(p_path, in_dir, k)} {os.path.join(p_path, out_train_dir)}
        else:
            if not os.path.isfile(os.path.join(p_path, out_test_dir, k)):
                !copy {os.path.join(p_path, in_dir, k)} {os.path.join(p_path, out_test_dir)}

save_train_test(project_path, dir_trimmed_video, dir_train, dir_test, video_train_test_dict)

        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 fi

In [10]:
####################### DO NOT NEED TO RERUN THIS CELL #######################
# Save training and validation videos in the same folder
def save_train_test2(p_path, in_dir, out_train_test_dir, in_train_test_dict):
    for k, v in in_train_test_dict.items():
        if v == 'train':
            video_name = k[:-4] + '_train.mov'
            if not os.path.isfile(os.path.join(p_path, out_train_test_dir, video_name)):
                !copy {os.path.join(p_path, in_dir, k)} {os.path.join(p_path, out_train_test_dir, video_name)}
        else:
            video_name = k[:-4] + '_test.mov'
            if not os.path.isfile(os.path.join(p_path, out_train_test_dir, video_name)):
                !copy {os.path.join(p_path, in_dir, k)} {os.path.join(p_path, out_train_test_dir, video_name)}

save_train_test2(project_path, dir_trimmed_video, dir_train_test, video_train_test_dict)

        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 file(s) copied.
        1 fi

* ASL_2006_10_10_scene3-camera1_BASEBALL_v1_c1_train.mov
* ASL_2006_10_10_scene3-camera1_AWKWARD_v0_c4_test.mov
* ASL_2007_05_24_scene12-camera1_DRESS_v0_c18_train.mov
* ASL_2007_05_24_scene12-camera1_ENOUGH_v1_c5_train.mov

In [13]:
# ASL_2006_10_10_scene3-camera1_BASEBALL_v1_c1_train.mov
# Actual 1 frame detected by cv2 are actually 2 frames by the video maker
# If I want to get the 100th frame per meta data, I need to get the 50th frame from the video.
original_video = 'ASL_2006_10_10_scene3-camera1.mov'
special_out_video = 'ASL_2006_10_10_scene3-camera1_BASEBALL_v1_c1_train.mov'
trim_one_video (project_path, dir_original_video, original_video, dir_train_test, special_out_video, int(1314/2), int(1337/2))

In [14]:
# ASL_2006_10_10_scene3-camera1_AWKWARD_v0_c4_test.mov
# same problem
original_video = 'ASL_2006_10_10_scene3-camera1.mov'
special_out_video = 'ASL_2006_10_10_scene3-camera1_AWKWARD_v0_c4_test.mov'
trim_one_video (project_path, dir_original_video, original_video, dir_train_test, special_out_video, int(1026/2), int(1089/2))

In [17]:
# ASL_2007_05_24_scene12-camera1_DRESS_v0_c18_train.mov
# same problem
original_video = 'ASL_2007_05_24_scene12-camera1.mov'
special_out_video = 'ASL_2007_05_24_scene12-camera1_DRESS_v0_c18_train.mov'
trim_one_video (project_path, dir_original_video, original_video, dir_train_test, special_out_video, int(192/4), int(231/4))


In [18]:
# ASL_2007_05_24_scene12-camera1_ENOUGH_v1_c5_train.mov
# same problem
original_video = 'ASL_2007_05_24_scene12-camera1.mov'
special_out_video = 'ASL_2007_05_24_scene12-camera1_ENOUGH_v1_c5_train.mov'
trim_one_video (project_path, dir_original_video, original_video, dir_train_test, special_out_video, int(2260/4), int(2301/4))
